In [ ]:
TODO:
#Use player scores to inform priors
#Use first few weeks to assess "market confidence"
#Finish Cumulative Season Stat Generation
#Other Measures of defensive prowess?
#Use Previous Years of Data

#Incorporate data from:
#- Sequential Home/Away
#- Player Scores
#- Injuries
#- Time Zones
#- Weather

Currently tests two baseline models for the 2017 season.  Will update with more data as it is acquired.

First baseline is an average of the pointspread predictions from various casinos and betting outlets. This does not require training, though we could train a ridge regression model and expect a similar result (as we'd expect the betting outlets to be relatively equally predictive on average as well as fairly correlated, so their weights should be very close under L2-regularized linear regression).

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Import data
odds = pd.read_csv("base_data/nfl_odds_2017.csv")
game_stats = pd.read_csv("base_data/nflstats2017.csv")
game_stats.rename(index = str, columns = {'HPS': 'APS', 'HPSY': 'APSY', 'HPS.1':'HPS', 'HPSY.1':'HPSY'}, inplace = True)

In [3]:
#Formatting to create key and filter
odds["datestring"] = pd.to_datetime(odds['date']).dt.strftime('%Y%m%d')
odds["Spread_key"] = odds['datestring'] + odds['Home']
odds['Spread_val'] = odds['HomeScore'] - odds['AwayScore']
spreads = odds.filter(regex = "Spread_")

#Compute square loss over 2017 season
X = spreads.values[:,:-2].astype('float32')
y = spreads['Spread_val'].values.astype('float32')
scores = np.nanmean(X, axis = 1)
residual = (scores - y)**2
loss = np.nanmean(residual)
print(loss)

347.285


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


Second baseline is a regression tree model that will use basic categorical features (Home/Away team, start time, home/away team stats) to predict the outcome.  For current purposes, will train on first 8 weeks of the season and test on the rest (this will change with more data).  This requires a lot more preprocessing, as we need to aggregate average statistics at that point in the season for each team.

In [62]:
def weekly_team_stats(stats, team):
    cols = ['Key','Pts','FD','Fum','FumL','PY','PA','PI','PS', 'PSY', 'RA', 'RY']
    t_stats = stats.query('Home == @team | Away == @team')
    weekly_stats = pd.DataFrame(columns = cols)
    count = 0
    
    for index,week in t_stats.iterrows():
        week = week.to_dict()
        
        if team == week['Home']:
            prefix = 'H'
            week.pop('Home')
        else:
            prefix = 'A'
            week.pop('Away')
        
        stats = {k[1:]: v for k, v in week.items() if k[0] == prefix}
        stats['Key'] = week['Key']
        weekly_stats = weekly_stats.append(stats, ignore_index = True)
        
    return weekly_stats

In [ ]:
def avg_team_stats(weekly_stats):
    cols = ['Key','C_Pts','C_FD','C_Fum','C_FumL','C_PY','C_PA','C_PI','C_PS', 'C_PSY', 'C_RA', 'C_RY']
    avg_stats = weekly_stats[]
    for index,week in 

In [63]:
game_stats['Datetime'] = pd.to_datetime(game_stats['Start'])
game_stats['Key'] = game_stats['Datetime'].dt.strftime('%Y%m%d') + game_stats['Home']
game_stats['Spread'] = game_stats['HPts'] - game_stats['APts']

teams = game_stats['Home'].unique()
team_stats = {}
for t in teams:
    team_stats.update({t:weekly_team_stats(game_stats, t)})
    


    


{'NWE':             Key Pts  FD Fum FumL   PY  PA PI PS PSY  RA   RY
0   20170907NWE  27  25   1    0  247  36  0  3  20  35  124
1   20170917NOR  36  29   0    0  436  39  0  2  11  31  119
2   20170924NWE  36  21   4    1  337  35  0  5  41  20   59
3   20171001NWE  30  24   0    0  293  45  0  3  14  19   80
4   20171005TAM  19  23   1    1  289  40  1  3  14  23  113
5   20171015NYJ  24  23   1    1  257  38  1  0   0  25  118
6   20171022NWE  23  25   1    0  241  29  0  2   8  36  162
7   20171029NWE  21  27   0    0  317  47  0  3  16  32   97
8   20171112DEN  41  28   0    0  297  37  0  1   6  29   99
9   20171119OAK  33  25   1    0  331  37  0  1   8  20   89
10  20171126NWE  35  25   4    1  221  28  1  1   6  38  196
11  20171203BUF  23  23   0    0  244  30  1  3  14  35  191
12  20171211MIA  20  14   0    0  223  43  2  2  10  10   25
13  20171217PIT  27  21   0    0  283  35  1  2  15  19   77
14  20171224NWE  37  28   0    0  218  28  1  2   6  38  193
15  20171231NWE 

Index(['Season', 'Start', 'Week', 'Away', 'Home', 'APts', 'HPts', 'OverUnder',
       'VegasLine', 'AFD', 'AFum', 'AFumL', 'APY', 'APA', 'API', 'APS', 'APSY',
       'ARA', 'ARY', 'HFD', 'HFum', 'HFumL', 'HPY', 'HPA', 'HPI', 'HPS',
       'HPSY', 'HRA', 'HRY'],
      dtype='object')
